In [1]:
!xtb --version


      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.7.0 (08769fc) compiled by 'albert@albert-system' on 2024-03-04

normal termination of xtb


In [2]:
import os, sys
import inspect
from pathlib import Path
import tempfile
from utils import run_command
import rdkit
from rdkit.Chem import AllChem, DataStructs,Draw
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

import numpy as np
import torch
import torch.nn as nn

In [4]:
smiles='C#CC#C'
m = Chem.MolFromSmiles('c1nccc2n1ccc2')
AllChem.Compute2DCoords(m)


0

In [5]:
template = Chem.MolFromSmiles('c1nccc2n1ccc2')
AllChem.Compute2DCoords(template)
ms = [Chem.MolFromSmiles(smi) for smi in ('OCCc1ccn2cnccc12','C1CC1Oc1cc2ccncn2c1','CNC(=O)c1nccc2cccn12')]
for m in ms:
    M = AllChem.GenerateDepictionMatching2DStructure(m,template)
M

((0, 4), (1, 5), (2, 6), (3, 7), (4, 8), (5, 12), (6, 11), (7, 10), (8, 9))

In [6]:

with Chem.SDMolSupplier('cdk2.sdf') as suppl:
    ms = [x for x in suppl if x is not None]
for m in ms: tmp=AllChem.Compute2DCoords(m)
m1=Draw.MolToFile(ms[0],'cdk2_mol1.png')    # doctest: +SKIP
m2=Draw.MolToFile(ms[1],'cdk2_mol2.png')    # doctest: +SKIP

OSError: File error: Bad input file cdk2.sdf

In [7]:
m2

NameError: name 'm2' is not defined

In [8]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D

# Fonction pour colorer les atomes donneurs et accepteurs
def color_atoms(mol):
    # Liste des donneurs et accepteurs communs
    donneurs = ['N', 'O']  # Atomes donneurs typiques
    accepteurs = ['N', 'O', 'F', 'Cl', 'Br', 'I']  # Atomes accepteurs typiques
    
    highlight_atoms = []
    atom_colors = {}
    
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        idx = atom.GetIdx()
        if symbol in donneurs:
            highlight_atoms.append(idx)
            atom_colors[idx] = (0, 0, 1)  # Bleu pour les donneurs
        if symbol in accepteurs:
            highlight_atoms.append(idx)
            atom_colors[idx] = (1, 0, 0)  # Rouge pour les accepteurs
    
    return highlight_atoms, atom_colors

# Exemple de SMILES
smiles = "c1ncc(-c2sc(-c3ccc(s3)-c3ncncn3)c3[SiH2]ccc23)c2nsnc12"  # Acide aminé alanine par exemple
mol = Chem.MolFromSmiles(smiles)

# Colorier les atomes
highlight_atoms, atom_colors = color_atoms(mol)

# Dessiner la molécule avec les atomes colorés
drawer = rdMolDraw2D.MolDraw2DCairo(300, 300)
drawer.DrawMolecule(mol, highlightAtoms=highlight_atoms, highlightAtomColors=atom_colors)
drawer.FinishDrawing()
drawer.WriteDrawingText("colored_molecule.png")

print("Molecule image saved as colored_molecule.png")


Molecule image saved as colored_molecule.png


In [9]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

def is_donor(mol):
    """
    Identifie si la molécule contient des groupes fonctionnels donneurs d'électrons.
    """
    donor_smarts = ['[NH2]', '[OCH3]', '[OH]']
    for smarts in donor_smarts:
        if mol.HasSubstructMatch(Chem.MolFromSmarts(smarts)):
            return True
    return False

def is_acceptor(mol):
    """
    Identifie si la molécule contient des groupes fonctionnels accepteurs d'électrons.
    """
    acceptor_smarts = ['[NO2]', '[CN]']
    for smarts in acceptor_smarts:
        if mol.HasSubstructMatch(Chem.MolFromSmarts(smarts)):
            return True
    return False

def analyze_molecule(smiles):
    """
    Analyse une molécule donnée par sa chaîne SMILES et renvoie certains descripteurs.
    """
    mol = Chem.MolFromSmiles(smiles)
    donor = is_donor(mol)
    acceptor = is_acceptor(mol)
    homo_lumo_gap = rdMolDescriptors.CalcNumHBA(mol) - rdMolDescriptors.CalcNumHBD(mol)
    descriptors = {
        'Molecular Weight': Descriptors.MolWt(mol),
        'HOMO-LUMO Gap': homo_lumo_gap,
        'Is Donor': donor,
        'Is Acceptor': acceptor
    }
    return descriptors

# Exemple de SMILES
smiles_list = ['C#CC#C', 'CC=O', 'CC(C)=O', 'C1=CC=C(C=C1)N', 'C1=CC=C(C=C1)O']
for smiles in smiles_list:
    properties = analyze_molecule(smiles)
    print(f'SMILES: {smiles}, Properties: {properties}')

SMILES: C#CC#C, Properties: {'Molecular Weight': 50.059999999999995, 'HOMO-LUMO Gap': 0, 'Is Donor': False, 'Is Acceptor': False}
SMILES: CC=O, Properties: {'Molecular Weight': 44.053, 'HOMO-LUMO Gap': 1, 'Is Donor': False, 'Is Acceptor': False}
SMILES: CC(C)=O, Properties: {'Molecular Weight': 58.080000000000005, 'HOMO-LUMO Gap': 1, 'Is Donor': False, 'Is Acceptor': False}
SMILES: C1=CC=C(C=C1)N, Properties: {'Molecular Weight': 93.12899999999999, 'HOMO-LUMO Gap': 0, 'Is Donor': True, 'Is Acceptor': False}
SMILES: C1=CC=C(C=C1)O, Properties: {'Molecular Weight': 94.11299999999997, 'HOMO-LUMO Gap': 0, 'Is Donor': True, 'Is Acceptor': False}


* SMILES: C#CC#C, Properties: {'Molecular Weight': 50.058, 'HOMO-LUMO Gap': 0, 'Is Donor': False, 'Is Acceptor': False}
* SMILES: CC=O, Properties: {'Molecular Weight': 44.053, 'HOMO-LUMO Gap': 1, 'Is Donor': False, 'Is Acceptor': True}
* SMILES: CC(C)=O, Properties: {'Molecular Weight': 58.079, 'HOMO-LUMO Gap': 1, 'Is Donor': False, 'Is Acceptor': True}
* SMILES: C1=CC=C(C=C1)N, Properties: {'Molecular Weight': 93.129, 'HOMO-LUMO Gap': 1, 'Is Donor': True, 'Is Acceptor': False}
* SMILES: C1=CC=C(C=C1)O, Properties: {'Molecular Weight': 94.112, 'HOMO-LUMO Gap': 1, 'Is Donor': True, 'Is Acceptor': False}

In [10]:
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
charge = Chem.rdmolops.GetFormalCharge(mol)
atom_number = mol.GetNumAtoms()
# Write the smile to a file
system = lambda x: run_command(x, verbose=5)

In [11]:

with open('test.smi', 'w') as f:
    f.write(smiles)

In [12]:

system('obabel test.smi --gen3D -O test.xyz')


1 molecule converted


input_dir = os.path.join(os.getcwd(), 'input_dir')
os.makedirs(input_dir, exist_ok=True)

  # Dossier contenant les fichiers .xyz d'origine
output_dir = 'okay'  # Dossier où seront créés les nouveaux fichiers .xyz
input_file= 'test.xyz'  # Nom du fichier CSV pour le dataframe
max_files = 20885  # Nombre maximum de fichiers à traiter

import subprocess
import os

def run_crest(input_file, output_dir, crest_path='crest'):
    """
    Fonction pour exécuter CREST 3.0 sur un fichier d'entrée et stocker les résultats dans un répertoire de sortie.
    
    :param input_file: Chemin vers le fichier de coordonnées d'entrée (format XYZ).
    :param output_dir: Répertoire où les résultats seront stockés.
    :param crest_path: Chemin vers l'exécutable CREST (défaut est 'crest' si dans le PATH).
    """
    # Assurez-vous que le répertoire de sortie existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Construire la commande CREST
    command = [crest_path, input_file, '--output', output_dir]
    
    # Exécution de la commande
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("CREST a été exécuté avec succès.")
        print("Sortie standard:\n", result.stdout.decode())
        print("Erreur standard:\n", result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de l'exécution de CREST: {e}")
        print("Sortie standard:\n", e.stdout.decode())
        print("Erreur standard:\n", e.stderr.decode())

if __name__ == '__main__':
    # Exemple d'utilisation
    input_xyz = 'molecule.xyz'  # Chemin vers votre fichier XYZ
    output_directory = 'crest_output'  # Répertoire de sortie
    crest_executable = 'crest'  # Chemin vers l'exécutable CREST si nécessaire

    run_crest(input_xyz, output_directory, crest_executable)

In [ ]:
!xtb --version

: 

In [12]:

command_pre = f'xtb test.xyz --gfn 2 --gbsa  h2o --opt normal -c {charge} --iterations 4000'
system(command_pre)

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.7.0 (08769fc) compiled by 'albert@albert-system' on 2024-03-04

   xtb is free software: you can redistribute it and/or modify it under
   the terms of the GNU Lesser General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.
   
   xtb is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY;

normal termination of xtb


In [13]:

command_crest = f'crest xtbopt.xyz --gfn2 --gbsa h2o --chrg {charge} -T 8'
system(command_crest)

 
       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.1, Mon May  6 18:43:33 UTC 2024
       commit (1782d7d) compiled by 'runner@fv-az772-53'
 
   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.
   • P.Pracht, S.Grimme, C.Bannwarth, F.Bohle, S.Ehlert,
     G.Feldmann, J.Gorges, M.Müller, T.Neudecker, C.Plett,
     S.Spicher, P.Steinbach, P.Wesołowski, F.Zeller,
     J. Chem. Phys., 2024, 160, 11

Un changement de topologie a été observé lors de l'optimisation initiale de la géométrie.
Cela pourrait être un artefact du niveau de théorie choisi (par exemple xTB).
Vous pouvez vérifier la trajectoire d'optimisation dans le fichier "crestopt.log".
Essayez l'une de ces options :

stata
Copier
A) Pré-optimisez votre entrée séparément et utilisez la structure optimisée
   comme entrée pour CREST. (Recommandé uniquement si la structure est intacte)

B) Redémarrez le même appel CREST que précédemment, mais ignorez le changement de topologie
   en utilisant le mot-clé "--noreftopo". (Peut produire des artefacts)

C) Corrigez la géométrie d'entrée initiale en introduisant des contraintes de longueur de liaison
   ou en utilisant une méthode avec une topologie fixe (par exemple GFN-FF).

In [13]:
!crest test.xyz --input input.toml


 
       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.1, Mon May  6 18:43:33 UTC 2024
       commit (1782d7d) compiled by 'runner@fv-az772-53'
 
   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.
   • P.Pracht, S.Grimme, C.Bannwarth, F.Bohle, S.Ehlert,
     G.Feldmann, J.Gorges, M.Müller, T.Neudecker, C.Plett,
     S.Spicher, P.Steinbach, P.Wesołowski, F.Zeller,
     J. Chem. Phys., 2024, 160, 11